In [2]:
import numpy as np
import pandas as pd
import requests
import bs4 as bs
import urllib.request

In [3]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2020"
source = urllib.request.urlopen(link).read()
soup = bs.BeautifulSoup(source, 'lxml')

In [4]:
tables = soup.find_all('table', class_ = 'wikitable sortable')

In [5]:
df1 = pd.read_html(str(tables[0]))[0]
df2 = pd.read_html(str(tables[1]))[0]
df3 = pd.read_html(str(tables[2]))[0]
df4 = pd.read_html(str(tables[3]).replace("'1\"\'",'"1"'))[0]

/var/folders/78/67qsv_654q11y5bbd4d1vb200000gn/T/ipykernel_3106/2807297294.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df1 = pd.read_html(str(tables[0]))[0]
/var/folders/78/67qsv_654q11y5bbd4d1vb200000gn/T/ipykernel_3106/2807297294.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df2 = pd.read_html(str(tables[1]))[0]
/var/folders/78/67qsv_654q11y5bbd4d1vb200000gn/T/ipykernel_3106/2807297294.py:3: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df3 = pd.read_html(str(tables[2]))[0]
/var/folders/78/67qsv_654q11y5bbd4d1vb200000gn/T/ipykernel_3106/2807297294.py:4: FutureWarning: Passing literal html to 'read_html

In [6]:
df = pd.concat([df1, pd.concat([df2, pd.concat([df3, df4], ignore_index = True)], ignore_index = True)], ignore_index = True)
df

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,J A N U A R Y,3,The Grudge,Screen Gems / Stage 6 Films / Ghost House Pict...,Nicolas Pesce (director/screenplay); Andrea Ri...,[2]
1,J A N U A R Y,10,Underwater,20th Century Fox / TSG Entertainment / Chernin...,"William Eubank (director); Brian Duffield, Ada...",[3]
2,J A N U A R Y,10,Like a Boss,Paramount Pictures / Artists First,"Miguel Arteta (director); Sam Pitman, Adam Col...",[4]
3,J A N U A R Y,10,Three Christs,IFC Films,Jon Avnet (director/screenplay); Eric Nazarian...,NaN
4,J A N U A R Y,10,Inherit the Viper,Lionsgate / Barry Films / Tycor International ...,Anthony Jerjen (director); Andrew Crabtree (sc...,[5]
...,...,...,...,...,...,...
273,D E C E M B E R,25,We Can Be Heroes,Netflix / Troublemaker Studios / Double R Prod...,Robert Rodriguez (director/screenplay); Priyan...,[246]
274,D E C E M B E R,25,News of the World,Universal Pictures / Playtone / Perfect World ...,Paul Greengrass (director/screenplay); Luke Da...,[247]
275,D E C E M B E R,25,One Night in Miami...,Amazon Studios,Regina King (director); Kemp Powers (screenpla...,[248]
276,D E C E M B E R,25,Promising Young Woman,Focus Features / FilmNation Entertainment,Emerald Fennell (director/screenplay); Carey M...,[249]


In [7]:
df_2020 = df[['Title', 'Cast and crew']]
df_2020['Title'] = df_2020['Title'].astype(str)

/var/folders/78/67qsv_654q11y5bbd4d1vb200000gn/T/ipykernel_3106/1633235931.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2020['Title'] = df_2020['Title'].astype(str)


In [8]:
df_2020

,Title,Cast and crew
0,The Grudge,Nicolas Pesce (director/screenplay); Andrea Ri...
1,Underwater,"William Eubank (director); Brian Duffield, Ada..."
2,Like a Boss,"Miguel Arteta (director); Sam Pitman, Adam Col..."
3,Three Christs,Jon Avnet (director/screenplay); Eric Nazarian...
4,Inherit the Viper,Anthony Jerjen (director); Andrew Crabtree (sc...
...,...,...
273,We Can Be Heroes,Robert Rodriguez (director/screenplay); Priyan...
274,News of the World,Paul Greengrass (director/screenplay); Luke Da...
275,One Night in Miami...,Regina King (director); Kemp Powers (screenpla...
276,Promising Young Woman,Emerald Fennell (director/screenplay); Carey M...


In [9]:
!pip install tmdbv3api

In [10]:
from tmdbv3api import TMDb
import json
import requests
tmdb = TMDb()
tmdb.api_key = '7d0d645c09f193f20fdb0a6e4cd50663'

In [11]:
from tmdbv3api import Movie
from tmdbv3api import Search

tmdb_movie = Movie() 
tmdb_search = Search()

def get_genre(x):
    if not isinstance(x, str) or not x.strip():  # Skip empty or non-string values
        return np.NaN
    
    try:
        result = tmdb_search.movies({"query": x})
        if not result:
            return np.NaN
        
        movie_id = result[0].id
        movie_details = tmdb_movie.details(movie_id)
        genres = [g['name'] for g in movie_details.genres]
        return ', '.join(genres) if genres else np.NaN
        
    except Exception as e:
        print(f"Error processing '{x}': {str(e)}")
        return np.NaN

In [12]:
df_2020['genres'] = df_2020['Title'].apply(lambda x: get_genre((x)))

Error processing 'The Grudge': quote_from_bytes() expected bytes
Error processing 'Underwater': quote_from_bytes() expected bytes
Error processing 'Like a Boss': quote_from_bytes() expected bytes
Error processing 'Three Christs': quote_from_bytes() expected bytes
Error processing 'Inherit the Viper': quote_from_bytes() expected bytes
Error processing 'The Sonata': quote_from_bytes() expected bytes
Error processing 'The Murder of Nicole Brown Simpson': quote_from_bytes() expected bytes
Error processing 'Angels Fallen': quote_from_bytes() expected bytes
Error processing 'Bad Boys for Life': quote_from_bytes() expected bytes
Error processing 'Dolittle': quote_from_bytes() expected bytes
Error processing 'A Fall from Grace': quote_from_bytes() expected bytes
Error processing 'The Gentlemen': quote_from_bytes() expected bytes
Error processing 'The Turning': quote_from_bytes() expected bytes
Error processing 'The Last Full Measure': quote_from_bytes() expected bytes
Error processing 'John He

/var/folders/78/67qsv_654q11y5bbd4d1vb200000gn/T/ipykernel_3106/1497638351.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2020['genres'] = df_2020['Title'].apply(lambda x: get_genre((x)))


In [13]:
df_2020

,Title,Cast and crew,genres
0,The Grudge,Nicolas Pesce (director/screenplay); Andrea Ri...,NaN
1,Underwater,"William Eubank (director); Brian Duffield, Ada...",NaN
2,Like a Boss,"Miguel Arteta (director); Sam Pitman, Adam Col...",NaN
3,Three Christs,Jon Avnet (director/screenplay); Eric Nazarian...,NaN
4,Inherit the Viper,Anthony Jerjen (director); Andrew Crabtree (sc...,NaN
...,...,...,...
273,We Can Be Heroes,Robert Rodriguez (director/screenplay); Priyan...,NaN
274,News of the World,Paul Greengrass (director/screenplay); Luke Da...,NaN
275,One Night in Miami...,Regina King (director); Kemp Powers (screenpla...,NaN
276,Promising Young Woman,Emerald Fennell (director/screenplay); Carey M...,NaN


In [14]:
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]

In [39]:
df_2020['director_name'] = df_2020['Cast and crew'].map(lambda x : get_director(str(x)))

/var/folders/78/67qsv_654q11y5bbd4d1vb200000gn/T/ipykernel_3106/1563575365.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2020['director_name'] = df_2020['Cast and crew'].map(lambda x : get_director(str(x)))


In [41]:
def get_actor1(x):
    return ((x.split("screenplay);")[-1]).split(', ')[0])

In [43]:
df_2020['actor_1_name'] = df_2020['Cast and crew'].map(lambda x : get_actor1(str(x)))

/var/folders/78/67qsv_654q11y5bbd4d1vb200000gn/T/ipykernel_3106/1264655752.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2020['actor_1_name'] = df_2020['Cast and crew'].map(lambda x : get_actor1(str(x)))


In [45]:
def get_actor2(x):
    if len((x.split('screenplay);')[-1]).split(', ')) < 2:
        return np.NAN
    else:
        return ((x.split('screenplay);')[-1]).split(', ')[1])

In [47]:
df_2020['actor_2_name'] = df_2020['Cast and crew'].map(lambda x : get_actor2(str(x)))

In [49]:
def get_actor3(x):
    if len((x.split('screenplay);')[-1]).split(', ')) < 3:
        return np.NAN
    else:
        return ((x.split('screenplay);')[-1]).split(', ')[2])

In [51]:
df_2020['actor_3_name'] = df_2020['Cast and crew'].map(lambda x : get_actor3(str(x)))

In [53]:
df_2020

,Title,Cast and crew,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,The Grudge,Nicolas Pesce (director/screenplay); Andrea Ri...,NaN,Nicolas Pesce,Andrea Riseborough,Demián Bichir,John Cho
1,Underwater,"William Eubank (director); Brian Duffield, Ada...",NaN,William Eubank,Kristen Stewart,Vincent Cassel,Jessica Henwick
2,Like a Boss,"Miguel Arteta (director); Sam Pitman, Adam Col...",NaN,Miguel Arteta,Tiffany Haddish,Rose Byrne,Salma Hayek
3,Three Christs,Jon Avnet (director/screenplay); Eric Nazarian...,NaN,Jon Avnet,Richard Gere,Peter Dinklage,Walton Goggins
4,Inherit the Viper,Anthony Jerjen (director); Andrew Crabtree (sc...,NaN,Anthony Jerjen,Josh Hartnett,Margarita Levieva,Chandler Riggs
...,...,...,...,...,...,...,...
273,We Can Be Heroes,Robert Rodriguez (director/screenplay); Priyan...,NaN,Robert Rodriguez,Priyanka Chopra Jonas,Pedro Pascal,YaYa Gosselin
274,News of the World,Paul Greengrass (director/screenplay); Luke Da...,NaN,Paul Greengrass,Tom Hanks,Helena Zengel,NaN
275,One Night in Miami...,Regina King (director); Kemp Powers (screenpla...,NaN,Regina King,Kingsley Ben-Adir,Eli Goree,Aldis Hodge
276,Promising Young Woman,Emerald Fennell (director/screenplay); Carey M...,NaN,Emerald Fennell,Carey Mulligan,Bo Burnham,Alison Brie


In [55]:
df_2020 = df_2020.rename(columns = {'Title' : 'movie_title'})

In [57]:
new_df20 = df_2020.loc[:,['director_name', 'actor_1_name', 'actor_2_name', 'actor_3_name', 'genres', 'movie_title']]

In [59]:
new_df20['comb'] = new_df20['actor_1_name'] + ' ' + new_df20['actor_2_name'] + ' ' + new_df20['actor_3_name'] + ' ' + new_df20['director_name'] + ' ' + new_df20['genres']

In [61]:
new_df20.isna().sum()

director_name      0
actor_1_name       0
actor_2_name       5
actor_3_name      28
genres           278
movie_title        0
comb             278
dtype: int64

In [63]:
new_df20 = new_df20.dropna(how = 'any')

In [65]:
new_df20.isna().sum()

director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           0
movie_title      0
comb             0
dtype: int64

In [67]:
new_df20['movie_title'] = new_df20['movie_title'].str.lower()

In [69]:
new_df20

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb


In [71]:
old_df = pd.read_csv('/Users/arunabhapani/Desktop/netflix_movie_recommendation_system/final_data.csv')

In [73]:
old_df

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,CCH Pounder Joel David Moore Wes Studi James C...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: episode vii - the force awakens ...,Doug Walker Rob Walker unknown Doug Walker Doc...
...,...,...,...,...,...,...,...
5858,"Nick Bruno, Troy Quane",Will Smith,Tom Holland,Tom Holland,Animation Action Adventure Comedy Family,spies in disguise,"Will Smith Tom Holland Tom Holland Nick Bruno,..."
5859,Greta Gerwig,Saoirse Ronan,Emma Watson,Florence Pugh,Drama Romance History,little women,Saoirse Ronan Emma Watson Florence Pugh Greta ...
5860,Sam Mendes,George MacKay,Dean-Charles Chapman,Mark Strong,War History Thriller Drama,1917,George MacKay Dean-Charles Chapman Mark Strong...
5861,Destin Daniel Cretton,Michael B. Jordan,Jamie Foxx,Jamie Foxx,Drama Crime History,just mercy,Michael B. Jordan Jamie Foxx Jamie Foxx Destin...


In [75]:
final_df = pd.concat([old_df, new_df20], ignore_index = True)

In [77]:
final_df

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,CCH Pounder Joel David Moore Wes Studi James C...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: episode vii - the force awakens ...,Doug Walker Rob Walker unknown Doug Walker Doc...
...,...,...,...,...,...,...,...
5858,"Nick Bruno, Troy Quane",Will Smith,Tom Holland,Tom Holland,Animation Action Adventure Comedy Family,spies in disguise,"Will Smith Tom Holland Tom Holland Nick Bruno,..."
5859,Greta Gerwig,Saoirse Ronan,Emma Watson,Florence Pugh,Drama Romance History,little women,Saoirse Ronan Emma Watson Florence Pugh Greta ...
5860,Sam Mendes,George MacKay,Dean-Charles Chapman,Mark Strong,War History Thriller Drama,1917,George MacKay Dean-Charles Chapman Mark Strong...
5861,Destin Daniel Cretton,Michael B. Jordan,Jamie Foxx,Jamie Foxx,Drama Crime History,just mercy,Michael B. Jordan Jamie Foxx Jamie Foxx Destin...


In [81]:
final_df.to_csv('main_data.csv', index = False)